## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-14-15-52-00 +0000,nypost,"$25 million taxpayer cash handed by DHS, FEMA ...",https://nypost.com/2025/08/14/world-news/d25-m...
1,2025-08-14-15-45-12 +0000,bbc,Air Canada begins cancelling flights ahead of ...,https://www.bbc.com/news/articles/cd9jy3299pjo...
2,2025-08-14-15-43-27 +0000,bbc,Criticism after Uefa displays 'stop killing ch...,https://www.bbc.com/sport/football/articles/cy...
3,2025-08-14-15-40-44 +0000,nypost,"ICE to roll out bold, new cars emblazoned with...",https://nypost.com/2025/08/14/us-news/ice-to-r...
4,2025-08-14-15-40-11 +0000,nyt,How Can the Government ‘Take Back’ a City It L...,https://www.nytimes.com/2025/08/14/upshot/wash...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2359/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
128,trump,55
79,putin,20
28,new,17
122,russia,13
134,court,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
67,2025-08-14-11-58-04 +0000,nypost,Russia appears ready to test new missile — eve...,https://nypost.com/2025/08/14/world-news/russi...,125
279,2025-08-13-17-57-20 +0000,nypost,Trump says he doesn’t believe he can convince ...,https://nypost.com/2025/08/13/us-news/trump-sa...,117
22,2025-08-14-14-50-16 +0000,nyt,Trump’s Affinity for Putin Will Be Tested at H...,https://www.nytimes.com/2025/08/13/us/politics...,109
232,2025-08-13-20-57-00 +0000,wsj,Trump Agrees on Ukraine Red Lines With Europe ...,https://www.wsj.com/world/trump-talks-ukraine-...,108
132,2025-08-14-08-52-43 +0000,nypost,Zelensky meets with UK’s Starmer as Europe bra...,https://nypost.com/2025/08/14/world-news/ukrai...,107


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
67,125,2025-08-14-11-58-04 +0000,nypost,Russia appears ready to test new missile — eve...,https://nypost.com/2025/08/14/world-news/russi...
144,43,2025-08-14-06-37-28 +0000,nypost,US fugitive Nicholas Rossi accused of faking h...,https://nypost.com/2025/08/14/us-news/fugitive...
279,42,2025-08-13-17-57-20 +0000,nypost,Trump says he doesn’t believe he can convince ...,https://nypost.com/2025/08/13/us-news/trump-sa...
166,39,2025-08-14-03-00-00 +0000,wsj,As other Asian countries ramp up manufacturing...,https://www.wsj.com/economy/trade/trump-tariff...
308,38,2025-08-13-16-03-42 +0000,nypost,"Trump announces George Strait, Sylvester Stall...",https://nypost.com/2025/08/13/us-news/trump-an...
103,38,2025-08-14-10-00-00 +0000,nypost,NY POSTcast Daily Debrief: Kohberger’s ‘living...,https://nypost.com/2025/08/14/us-news/ny-postc...
303,34,2025-08-13-16-25-00 +0000,wsj,New York Attorney General Letitia James is sui...,https://www.wsj.com/politics/policy/new-york-a...
74,33,2025-08-14-11-30-00 +0000,nypost,Eric Adams advances fight against NYC public d...,https://nypost.com/2025/08/14/us-news/eric-ada...
72,32,2025-08-14-11-39-23 +0000,wapo,"Top South Korean court listens to ‘Baby Shark,...",https://www.washingtonpost.com/style/2025/08/1...
1,31,2025-08-14-15-45-12 +0000,bbc,Air Canada begins cancelling flights ahead of ...,https://www.bbc.com/news/articles/cd9jy3299pjo...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
